In [1]:
import pandas as pd
import glob
import re
import csv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import simps
from numpy import trapz
from sklearn.metrics import auc
import math


paths = ["/home/seb300/Pong/learnability-Ttest",
         "/home/seb300/Pong/generalization-Ttest"]

#paths = ["/content/drive/MyDrive/master thesis/Pacman/generalization"]
file_dict={}
file_dict_results = {}

for path in paths:
  %cd $path
  method = path.split("/")[-1]
  if method not in file_dict:
    file_dict[method]={}
  for filename in glob.glob("./*"):
    # print(filename)
    #./learnability2agents_BoltzmannAgent_v3M_RandomGhost_{"index":1,"prob":{}}_{"mean":0,"std":0}_v3M_RandomGhost_{"index":1,"prob":{}}_{"mean":0,"std":0}.pkl
    elements = filename.replace(".pkl","").split("_")
    typeofagent = elements[1]
    envs = re.split(r'{"mean".*?}', "".join(elements[3:]))
    noises = re.findall(r'{"mean".*?}', filename)
    if "ensemble" in method:
      testenv = envs[0]
      noise_testenv = noises[0]
      try:
        trainenv = envs[1]
      except:
        trainenv = ""
      noise_trainenv = noises[1]
    elif "generalization" in method:
      try:
        testenv = envs[1]
      except:
        testenv = ""
      noise_testenv = noises[1]
      trainenv = envs[0]
      noise_trainenv = noises[0]
    else:
      testenv = envs[0]
      noise_testenv = noises[0]
      trainenv = envs[0]
      noise_trainenv = noises[0]
    if typeofagent not in file_dict[method]:
      file_dict[method][typeofagent] = {}
    if testenv not in file_dict[method][typeofagent]:
      file_dict[method][typeofagent][testenv] = {}
    if noise_testenv not in file_dict[method][typeofagent][testenv]:
      file_dict[method][typeofagent][testenv][noise_testenv] = {}
    if trainenv not in file_dict[method][typeofagent][testenv][noise_testenv]:
      file_dict[method][typeofagent][testenv][noise_testenv][trainenv] = {}
    if noise_trainenv not in file_dict[method][typeofagent][testenv][noise_testenv][trainenv]:
      file_dict[method][typeofagent][testenv][noise_testenv][trainenv][noise_trainenv] = {}
    with open(filename, newline='') as csvfile:
      if "errorbar" in filename:
        file_dict[method][typeofagent][testenv][noise_testenv][trainenv][noise_trainenv]["errorbar"] = []
      else:
        file_dict[method][typeofagent][testenv][noise_testenv][trainenv][noise_trainenv]["values"] = []
      reader = csv.reader(csvfile, delimiter='\n', quotechar='|')
      for row in reader:
        list_el = []
        for el in row[0].split(","):
          list_el.append(float(el))
        if "errorbar" in filename:
          file_dict[method][typeofagent][testenv][noise_testenv][trainenv][noise_trainenv]["errorbar"].append(float(row[0]))
        else:
          file_dict[method][typeofagent][testenv][noise_testenv][trainenv][noise_trainenv]["values"].append(float(row[0]))

/home/seb300/miniconda3/envs/.py39env/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/seb300/Pong/learnability-Ttest
/home/seb300/Pong/generalization-Ttest


In [2]:
import scipy
def TTest(x,y):
    return x.mean(), y.mean(), scipy.stats.ttest_ind(x,y)

In [3]:
# NON SEMANTIC NOISE
somegrid_l = ['pong','pong-thick']
someagent_l = ['BoltzmannAgent', 'SarsaAgent']
someexploration_strategy_l = ['Boltzmann', 'Egreedy']
#training env
somebar_l = ['ComputerBar{"index":1,"prob":{}}','DirectionalComputerBar{"index":1,"prob":0.3}','DirectionalComputerBar{"index":1,"prob":0.6}']
somenoise_l = ['{"mean":0,"std":0}','{"mean":0,"std":0}','{"mean":0,"std":0}']
#testing env
someotherbar_l = ['ComputerBar{"index":1,"prob":{}}','DirectionalComputerBar{"index":1,"prob":0.3}','DirectionalComputerBar{"index":1,"prob":0.6}']
someothernoise_l =  ['{"mean":0,"std":0.1}','{"mean":0,"std":0.1}','{"mean":0,"std":0.1}']

for someagent in someagent_l:
    for someexploration_strategy in someexploration_strategy_l:
        for somegrid in somegrid_l:
            for somebar, somenoise, someotherbar, someothernoise in zip(somebar_l, somenoise_l, someotherbar_l, someothernoise_l):
                print(someagent, someexploration_strategy, f"{somegrid}{somebar}", somenoise, f"{somegrid}{someotherbar}", someothernoise)
                G =  file_dict['generalization-Ttest'][someagent][f"{somegrid}{somebar}"][someothernoise][f"{somegrid}{somebar}"][somenoise]["values"]
                L =  file_dict['learnability-Ttest'][someagent][f"{somegrid}{somebar}"][someothernoise][f"{somegrid}{somebar}"][someothernoise]["values"]
                print(TTest(np.asarray(G),np.asarray(L)))          

BoltzmannAgent Boltzmann pongComputerBar{"index":1,"prob":{}} {"mean":0,"std":0} pongComputerBar{"index":1,"prob":{}} {"mean":0,"std":0.1}
(304.12979985809324, 308.91640038708334, TtestResult(statistic=-0.7205308175420574, pvalue=0.47131020150525127, df=1504.0))
BoltzmannAgent Boltzmann pongDirectionalComputerBar{"index":1,"prob":0.3} {"mean":0,"std":0} pongDirectionalComputerBar{"index":1,"prob":0.3} {"mean":0,"std":0.1}
(289.15939985466, 262.6080004964284, TtestResult(statistic=3.30808600881255, pvalue=0.0009728769910576257, df=999.0))
BoltzmannAgent Boltzmann pongDirectionalComputerBar{"index":1,"prob":0.6} {"mean":0,"std":0} pongDirectionalComputerBar{"index":1,"prob":0.6} {"mean":0,"std":0.1}
(232.7873998222351, 163.7224003592889, TtestResult(statistic=7.990288991564267, pvalue=3.693733642206185e-15, df=999.0))
BoltzmannAgent Boltzmann pong-thickComputerBar{"index":1,"prob":{}} {"mean":0,"std":0} pong-thickComputerBar{"index":1,"prob":{}} {"mean":0,"std":0.1}
(153.4814000606537, 2

In [4]:
# SEMANTIC NOISE
somegrid_l = ['pong','pong-thick']
someagent_l = ['BoltzmannAgent', 'SarsaAgent']
someexploration_strategy_l = ['Boltzmann', 'Egreedy']
#training env
somebar_l = ['DirectionalComputerBar{"index":1,"prob":0.3}','DirectionalComputerBar{"index":1,"prob":0.6}']
somenoise_l = ['{"mean":0,"std":0}','{"mean":0,"std":0}']
#testing env
someotherbar_l = ['ComputerBar{"index":1,"prob":{}}','ComputerBar{"index":1,"prob":{}}']
someothernoise_l =  ['{"mean":0,"std":0}','{"mean":0,"std":0}']

for someagent in someagent_l:
    for someexploration_strategy in someexploration_strategy_l:
        for somegrid in somegrid_l:
            for someotherbar, somenoise, someotherbar, someothernoise in zip(someotherbar_l, somenoise_l, someotherbar_l, someothernoise_l):
                print(someagent, someexploration_strategy, f"{somegrid}{someotherbar}", somenoise, f"{somegrid}{someotherbar}", someothernoise)
                G =  file_dict['generalization-Ttest'][someagent][f"{somegrid}{someotherbar}"][someothernoise][f"{somegrid}{someotherbar}"][somenoise]["values"]
                L =  file_dict['learnability-Ttest'][someagent][f"{somegrid}{someotherbar}"][someothernoise][f"{somegrid}{someotherbar}"][someothernoise]["values"]
                print(TTest(np.asarray(G),np.asarray(L)))          

BoltzmannAgent Boltzmann pongComputerBar{"index":1,"prob":{}} {"mean":0,"std":0} pongComputerBar{"index":1,"prob":{}} {"mean":0,"std":0}
(450.2546003112793, 450.92754627813395, TtestResult(statistic=-0.2004297150325459, pvalue=0.8411786260458027, df=1195.0))
BoltzmannAgent Boltzmann pongComputerBar{"index":1,"prob":{}} {"mean":0,"std":0} pongComputerBar{"index":1,"prob":{}} {"mean":0,"std":0}
(450.2546003112793, 450.92754627813395, TtestResult(statistic=-0.2004297150325459, pvalue=0.8411786260458027, df=1195.0))
BoltzmannAgent Boltzmann pong-thickComputerBar{"index":1,"prob":{}} {"mean":0,"std":0} pong-thickComputerBar{"index":1,"prob":{}} {"mean":0,"std":0}
(447.8506004638672, 450.74961779651744, TtestResult(statistic=-0.9424132499925816, pvalue=0.34614220136301876, df=1413.0))
BoltzmannAgent Boltzmann pong-thickComputerBar{"index":1,"prob":{}} {"mean":0,"std":0} pong-thickComputerBar{"index":1,"prob":{}} {"mean":0,"std":0}
(447.8506004638672, 450.74961779651744, TtestResult(statistic